In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt


# Extracting the Features

In [2]:
img1 = cv2.imread('images/Fifty_thousand_Dram_2.jpg',0)          # queryImage
#img2 = cv2.imread('images/Ten_thousand_Dram_3.jpg',0) # trainImage

# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
bf = cv2.BFMatcher()

In [33]:
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    kp2, des2 = sift.detectAndCompute(grayframe,None)

    # BFMatcher with default params
   
    matches = bf.knnMatch(des1,des2, k=2)
    
    # Apply ratio test.
    
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])
    
    # cv2.drawMatchesKnn expects list of lists as matches.
    #img3 = cv2.drawMatchesKnn(img1,kp1,grayframe,kp2,good,None,flags=2)
    #Homograpy
    
    if len(good) > 10:
        query_pts = np.float32([kp2[i].pt for i in good]).reshape(-1, 1, 2)
        train_pts = np.float32([kp2[j].pt for j in good]).reshape(-1, 1, 2)
        
        
        matrix, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0)
        matches_mask = mask.ravel().tolist()
        
        
        # Perspective transform
        h, w = img.shape
        pts = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)
        dst = cv2.perspectiveTransform(pts, matrix)

    else:   
        cv2.imshow('Homo',grayframe) 
        
    #cv2.imshow('Frame', img3)
    
    #cv2.imshow('Frame', grayframe)
#    
    
    key = cv2.waitKey(1)
    if key == 27:
        break
    
    
cap.release()
cv2.destroyAllWindows()

TypeError: list indices must be integers or slices, not list

In [46]:
kp2[1]

<KeyPoint 000001AE9A9BDF60>

In [44]:
good

[[<DMatch 000001AE9B78ABD0>],
 [<DMatch 000001AE9B78AC50>],
 [<DMatch 000001AE9B78C990>],
 [<DMatch 000001AE9B790530>],
 [<DMatch 000001AE9B79C110>],
 [<DMatch 000001AE9B7ABC30>],
 [<DMatch 000001AE9B7ABFB0>],
 [<DMatch 000001AE9B7B09F0>],
 [<DMatch 000001AE9B7B0BF0>],
 [<DMatch 000001AE9B7B0C30>],
 [<DMatch 000001AE9B7C2BB0>],
 [<DMatch 000001AE9B7C43D0>],
 [<DMatch 000001AE9B7C4970>],
 [<DMatch 000001AE9B7C6170>],
 [<DMatch 000001AE9B7C6490>],
 [<DMatch 000001AE9AA72E90>],
 [<DMatch 000001AE9AA77190>],
 [<DMatch 000001AE9AA77890>],
 [<DMatch 000001AE9AA71B90>],
 [<DMatch 000001AE9AA81A10>],
 [<DMatch 000001AE9AA81E50>],
 [<DMatch 000001AE9AA858D0>],
 [<DMatch 000001AE9AA88810>],
 [<DMatch 000001AE9AA889D0>],
 [<DMatch 000001AE9AA88A50>],
 [<DMatch 000001AE9B7EFA10>],
 [<DMatch 000001AE9B7EFED0>],
 [<DMatch 000001AE9B7C8C10>],
 [<DMatch 000001AE9B7CB090>],
 [<DMatch 000001AE9B7CB150>],
 [<DMatch 000001AE9B7CB290>],
 [<DMatch 000001AE9B7CD9D0>],
 [<DMatch 000001AE9B7D1950>],
 [<DMatch 

In [41]:
query_pts = np.float32([kp2[m].pt for m in good]).reshape(-1, 1, 2)

TypeError: list indices must be integers or slices, not list